# Voedselprijzen in de derde wereld

Robert-Jan Korteschiel (10399143)  
Robert Houten  
Sander Kohnstamm (10715363)  
Joost de Wildt (12173002)  


## Vooronderzoek

Dit is een dataset die wekelijks wordt geupdated over voedselprijzen. De auteur is het World Food Programme, een humanitaire organisatie die steeft naar het ideaal van "zero hunger". Daar baseren ze al hun werk al op data en hebben ook al goede dataviz draaien. Interessant is dat ze hier combineren met veel GIS informatie zoals administratieve grenzen, wegennetwerken en klimaatdata. 

<div style="display: flex; width: 100%; justify-content: space-around;">
    <div style="width: 45%">
        <img src="food_viz.png" style="display: block; width: 100%; height: auto;" alt="GIS visualisation">
    </div>
    <div style="width: 45%">
        <img src="patterns.jpg" style="display: block; width: 100%; height: auto;" alt="Line visualisation">
    </div>
</div>


Er is dus al veel werk gedaan op de data, maar er mist ook veel informatie. Opvallend is dat economische en politieke gegevens missen in de analyse. Daarnaast bekijken ze de derde wereld echt per land en zien kijken ze bijna niet naar hoe deze problematiek grensoverstijgend is (of is dat wel zo?). Bovendien is het alles behalve een verhaal, dit is echt een visualisatie. Het is interessant of we delen kunnen namaken en tot een verhaal kunnen omvormen. 

## Invalshoeken

1. Welke van de volgende gebeurtenissen heeft de grootste invloed op globale voedselprijzen?
    - Temperatuur
    - Brandstofprijs
    - Dagloon  


2. Vallen gebeurtenissen te herleiden uit voedselprijzen?



# Food

## Bootstrap

In [87]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn import preprocessing
from operator import itemgetter, attrgetter
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from functools import partial

init_notebook_mode(connected=True)

pd.options.display.max_rows = 100
pd.options.display.max_seq_items = 100

## Inladen

en laat de head() zien

In [109]:
food_df = pd.read_csv("./food_data/food.csv", low_memory=False)
display(food_df.head())

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


## Transform

### Normalisatie

Normalisatie functie voor verschillende groepen in de data. Ik vermoed dat je het beste binnen een land op commodity kan normaliseren. Doe je het op een andere plek dan worden minder waardevolle commodities binnen groepen minder belangrijk. Het is aannemelijk dat graan wholesale flink goedkoper is dan graan retail. Als je die in absolute nummers optelt dan domineert het verschil in de retailprijs.

Je zou overigens nog een argument kunnen maken dat je eigenlijk op provincie of zelfs stadsniveau moet gaan normaliseren.


In [89]:
def normalize_group(key, group_df):
    # bootstrap a new normalizer   
    min_max_scaler = preprocessing.MinMaxScaler()
    
    # reshape the series to array so that the normalizer accepts it
    to_normalize = group_df[key].values.reshape(-1, 1)
    
    # do the actual normalisation     
    x_scaled = min_max_scaler.fit_transform(to_normalize)
    
    # undo some weird numpy nesting of arrays    
    x_scaled = np.concatenate(x_scaled).ravel()
    
    # concatenate it to the group_df    
    group_df[f"{key}_norm"] = x_scaled
    
    # trow the group out     
    return group_df
    
food_norm_df = food_df.groupby(by=["adm0_name", "cm_name"]).apply(partial(normalize_group, "mp_price"))

### Groeperen en verwerken

De vergelijkbaarheid per commodity per land echt heel laag, maar hele korte stukken data overlappen echt. Het WFP meet blijkbaar alleen als ze geinteresseerd zijn in iets, om de een of andere reden. Vervolgens redeneren we dat we al het voedsel samentrekken tot een enkele trend, dat moet de vergelijkbaarheid al aardig hoger maken. Voor onze onderzoeksvraag is het immers alleen interessant dat we veranderingen in voedselprijzen afzetten tegen brandstof, lonen en klimaat. Misschien dat we daarna gewoon een land kiezen met het meeste data, los van de vergelijkbaarheid, voor de andere onderzoeksvraag.


Groepeert alle commodities op basis het het eerste woord in hun definitie. Daarna berekent hij het gemiddelde en het verschil per jaar. 

**LET OP! Hier zit een grote assumptie. Een jaar kan een waarde krijgen op een enkele meting**

In [118]:
# groepeer de commodities met hun eerst woord
pat = '^([\w\-]+)'
selection = food_norm_df["cm_name"].str.extract(pat, expand=False)
grouped_commodity_df = food_norm_df.groupby(by=["adm0_name", selection, "mp_year"])

# bereken het gemiddelde per jaar en vergelijk dat met het vorige jaar
grouped_commodity_mean_df = grouped_commodity_df["mp_price_norm"].mean()

# unstack zodat het verschil per jaar berekend kan worden
grouped_commodity_diff_df = grouped_commodity_mean_df.unstack(level=2).diff(axis=1)
grouped_commodity_diff_df.head()

mp_year                1992  1993  1994  1995  1996  1997  1998  1999  2000  \
adm0_name   cm_name                                                           
Afghanistan Bread       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
            Exchange    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
            Fuel        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
            Livestock   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
            Rice        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

mp_year                    2001  ...      2010      2011      2012      2013  \
adm0_name   cm_name              ...                                           
Afghanistan Bread           NaN  ...       NaN       NaN       NaN       NaN   
            Exchange        NaN  ...       NaN       NaN       NaN       NaN   
            Fuel      -0.007045  ...       NaN       NaN       NaN       NaN   
            Livestock  0.135611  ...       NaN       NaN       NaN       NaN   
            Rice            NaN  ... -0.041588  0.015685  0.072097  0.123801   

mp_year                    2014      2015      2016      2017      2018  \
adm0_name   cm_name                                                       
Afghanistan Bread           NaN  0.055021 -0.007271 -0.006384 -0.004228   
            Exchange        NaN  0.178623  0.301038  0.014753  0.200153   
            Fuel            NaN -0.000093 -0.000046  0.000044  0.000035   
            Livestock       NaN       NaN       NaN       NaN       NaN   
            Rice      -0.054074 -0.047844  0.025185  0.038895  0.025792   

mp_year                    2019  
adm0_name   cm_name              
Afghanistan Bread      0.000900  
            Exchange   0.154888  
            Fuel       0.000014  
            Livestock       NaN  
            Rice      -0.002980  

[5 rows x 28 columns]

### Data selecteren 

Verkenning en functies om te selecteren.

Spoiler: De groep "Rice" heeft de meeste datapunten en 2016 is het jaar met de minste NaN's.

In [117]:
# een telling van hoeveelheid gevulde vakjes per commodity
years_with_data_per_country = [(commodity_type, grouped_commodity_diff_df.xs(commodity_type, level=1, drop_level=False).count().sum()) for commodity_type in selection.unique()]

# welke is het beste
uselection_sorted = sorted(years_with_data_per_country, key=itemgetter(1), reverse=True)

# maak het even makkelijk voor de visualisatie
uselection_sorted_comm = [comm[0] for comm in uselection_sorted]
uselection_sorted_comm_ten = uselection_sorted_comm[:10]
uselection_sorted_comm_ten

# Utility om selecteren makkelijker te maken (of moeilijker)
def select_commodity(commodity, year):
    commodity_series = grouped_commodity_diff_df.xs(commodity, level=1, drop_level=False)[year]
    commodity_index = commodity_series.index.get_level_values(0)
    commodity_values = commodity_series.values
    return [commodity_index, commodity_values]

## Map

### Food change maps


In [80]:
def map_graph(food=uselection_sorted_comm_ten, year=(2007, 2019, 1)):
    # Plot the mean price, of all commodities on map, of all years.
    com_index_value_pair = select_commodity(food, year)

    map_data = [go.Choropleth(
        locations = com_index_value_pair[0],
        locationmode = "country names",
        z = com_index_value_pair[1],
    )]

    fig = go.Figure(data = map_data)
    iplot(fig)
    
interact(map_graph, food=uselection_sorted_comm_ten, year=(2007, 2019, 1))


interactive(children=(Dropdown(description='food', options=('Rice', 'Maize', 'Oil', 'Wheat', 'Beans', 'Sugar',…

<function __main__.map_graph(food=['Rice', 'Maize', 'Oil', 'Wheat', 'Beans', 'Sugar', 'Sorghum', 'Meat', 'Millet', 'Potatoes'], year=(2007, 2019, 1))>

## Controle

Om te controleren of de dataset correct is moeten we natuurlijk even een test draaien. Ik heb globale voedselprijzen even als benchmark genomen. Daar ligt waarschijnlijk een sterke correlatie, zeker voor gebieden die niet direct in conflict zijn.

[rijstprijzen] https://www.finanzen.nl/grondstoffen/grafiek/rijstprijs  
[imf commodity prices data] https://www.imf.org/en/Research/commodity-prices




In [115]:
# selecteer rijst en bereken het gemiddelde
commodity_imf_df = pd.read_csv("./commodity_imf_proper.csv", dtype=float, decimal=",")
commodity_imf_df = commodity_imf_df.set_index('Year')

def normalize_series(series):
    # bootstrap a new normalizer   
    min_max_scaler = preprocessing.MinMaxScaler()
    
    # reshape the series to array so that the normalizer accepts it
    to_normalize = series.values.reshape(-1, 1)
    
    # do the actual normalisation     
    x_scaled = min_max_scaler.fit_transform(to_normalize)
    
    # undo some weird numpy nesting of arrays    
    x_scaled = np.concatenate(x_scaled).ravel()
    
    # trow the series     
    return pd.Series(x_scaled, index = series.index)

def line_graph(food):
    # create an average of all countries    
    test = grouped_commodity_diff_df.xs(food, level=1, drop_level=False).mean()
    
    # use the IMF data to compute a nomalized and diffed line too    
    control = normalize_series(commodity_imf_df[food]).diff()
    
    test_trace = go.Scatter(
        name = "WFP",
        x = test.index,
        y = test.values
    )
    
    control_trace = go.Scatter(
        name = "IMF",
        x = control.index[-19:],
        y = control.values[-19:]
    )
    
    iplot([test_trace, control_trace], filename='basic-line')

interact(line_graph, food=["Rice", "Wheat", "Sugar"])



interactive(children=(Dropdown(description='food', options=('Rice', 'Wheat', 'Sugar'), value='Rice'), Output()…

<function __main__.line_graph(food)>

## Resultaat

Er lijkt wel iets van een correlatie te zijn, maar er zitten ook rare verschillen tussen. Metname de pieken zijn veel hoger. Maar duidelijk is dat 2008 een grote impact heeft gehad. Ik denk dat we nog even goed moeten kijken naar hoe we normaliseren. Het is best mogelijk om hiermee te spelen lijkt het, maar wat verantwoord is twijfel ik aan.

# Klimaat

Nu eens kijken of het samenhangt met klimaat. Ik heb het gevoel dat hier enige correlatie vinden echt heel onmogelijk gaat worden. Die markten zijn veel te volatiel. Je moet daarvoor gaan compenseren om algemene trend te zien. Maar ik zal een poging doen.

## Load data

In [110]:

climate_df = pd.read_csv("./climate_data/GlobalLandTemperaturesByCountry.csv", low_memory=False)
display(climate_df.head())


,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


## Type and normalize

Here we go again.

1. Type the datetime
2. Normalize
3. Create year averages
4. Diff those averages to have something comparable

This is just a tad different data though. It's cumulative data, its not the same as money. And the effects are much longer term. Doesn't make sense.

In [112]:
# type properly
climate_df_typed = climate_df.copy()
climate_df_typed['dt'] = climate_df_typed['dt'].astype('datetime64[ns]')

# apply the normalize function (with a partial for the key)
climate_norm_df = climate_df_typed.groupby(by=["Country"]).apply(partial(normalize_group, "AverageTemperature"))


/Users/Robert-Jan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:355: RuntimeWarning:

All-NaN slice encountered

/Users/Robert-Jan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: RuntimeWarning:

All-NaN slice encountered



## Transform and select

In [113]:
# calulate the mean for the year
climate_mean_df = climate_norm_df.groupby(by=["Country", climate_norm_df["dt"].dt.year]).mean()
climate_mean_df.reset_index(inplace=True)

# quick function with assignment
def diff_temp(group_df):
    group_df["AverageTemperature_diff"] = group_df["AverageTemperature_norm"].diff()
    return group_df

# apply the difference function
climate_tran_df = climate_mean_df.groupby(by=["Country"]).apply(diff_temp).set_index(["Country", "dt"])

# quick selection function
def select_year(year):
    return climate_tran_df.xs(year, level=1, drop_level=False)

## Map

In [114]:
def map_climate(year):
    year_data = select_year(year)
    map_data = [go.Choropleth(
            locations = year_data.index.get_level_values(0),
            locationmode = "country names",
            z = year_data["AverageTemperature_diff"].values,
        )]

    fig = go.Figure(data = map_data)
    iplot(fig)
    
interact(map_climate, year=(2007, 2013, 1))




interactive(children=(IntSlider(value=2010, description='year', max=2013, min=2007), Output()), _dom_classes=(…

<function __main__.map_climate(year)>

# Bronnen

[datasource] https://data.humdata.org/dataset/wfp-food-prices  
[organisation goals] https://www1.wfp.org/zero-hunger  
[data description] https://docs.wfp.org/api/documents/WFP-0000040024/download/  
[dataviz platform] https://dataviz.vam.wfp.org  
[dataviz hunger] https://dataviz.vam.wfp.org/Hunger-Analytics-Hub#  
[WFP price] https://dataviz.vam.wfp.org/economic_explorer/price-forecasts-alerts   
[VN prijzen] https://treasury.un.org/operationalrates/OperationalRates.php  
[dataset omschrijving] http://mvam.org/2018/11/20/getting-up-to-speed-wfp-food-data-on-hdx/  
[interactieve plots] https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6  
[GDELT database] https://www.gdeltproject.org/  
[datastory] https://ourworldindata.org/food-prices  

